In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
df_coor = pd.read_csv("C:/Users/hejia/Desktop/DS_projects/TorontaData.csv")

In [3]:
df_coor["Borough"].value_counts().sum

<bound method Series.sum of North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Name: Borough, dtype: int64>

In [4]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [5]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_coor['Latitude'], df_coor['Longitude'], df_coor['Borough'], df_coor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

let us explore the neighborhoods in North York of Toronto

In [6]:
n_york_data = df_coor[df_coor['Borough'] == 'North York'].reset_index(drop=True)
n_york_data.head()

,Unnamed: 0,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
3,7,M3B,North York,Don Mills,43.745906,-79.352188
4,10,M6B,North York,Glencairn,43.709577,-79.445073


In [7]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [8]:
# create map of North York using latitude and longitude values
map_n_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(n_york_data['Latitude'], n_york_data['Longitude'], n_york_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_n_york)  
    
map_n_york

In [9]:
# define Foursquare Credentials and Version
CLIENT_ID = '3TJILUL33Y122CMLX3WN4H245RQZHXPF13MRJ5UWKTG4VS1Z' # your Foursquare ID
CLIENT_SECRET = 'RS2ZHMTYLMYOA40SHDRXXXXD521CGDDA1ZW1M3ZT3P15VLB5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [10]:
# create a function to repert exploring all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

n_york_venues = getNearbyVenues(names=n_york_data['Neighborhood'],
                                   latitudes=n_york_data['Latitude'],
                                   longitudes=n_york_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor / Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Fairview / Henry Farm / Oriole
Northwood Park / York University
Bayview Village
Downsview
York Mills / Silver Hills
Downsview
North Park / Maple Leaf Park / Upwood Park
Humber Summit
Willowdale / Newtonbrook
Downsview
Bedford Park / Lawrence Manor East
Humberlea / Emery
Willowdale
Downsview
York Mills West
Willowdale


In [12]:
n_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bathurst Manor / Wilson Heights / Downsview North,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,23,23,23,23,23,23
Don Mills,27,27,27,27,27,27
Downsview,16,16,16,16,16,16
Fairview / Henry Farm / Oriole,62,62,62,62,62,62
Glencairn,4,4,4,4,4,4
Hillcrest Village,5,5,5,5,5,5
Humber Summit,1,1,1,1,1,1


In [13]:
print('There are {} uniques categories.'.format(len(n_york_venues['Venue Category'].unique())))

There are 103 uniques categories.


In [14]:
# Analyze each Neighborhood
# one hot encoding
n_york_onehot = pd.get_dummies(n_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
n_york_onehot['Neighborhood'] = n_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [n_york_onehot.columns[-1]] + list(n_york_onehot.columns[:-1])
n_york_onehot = n_york_onehot[fixed_columns]

n_york_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
n_york_onehot.shape

(238, 104)

In [16]:
n_york_grouped = n_york_onehot.groupby('Neighborhood').mean().reset_index()
n_york_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Bathurst Manor / Wilson Heights / Downsview North,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.00,0.000000,0.050000,0.000000,0.000000,0.000000,0.05,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.05,0.0,0.05,0.050000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.050000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.050000,0.000000,0.000000,0.0,0.000000,0.000000,0.050000,0.050000,0.000000,0.0,0.0,0.000000,0.000000,0.050000,0.000000,0.050000,0.000000,0.050000,0.000000,0.000000,0.000000,0.050000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000
1,Bayview Village,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.00,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,Bedford Park / Lawrence Manor East,0.000000,0.0000,0.043478,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.043478,0.0,0.043478,0.000000,0.000000,0.000000,0.000000,0.086957,0.043478,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.00,0.0,0.00,0.000000,0.0,0.043478,0.043478,0.000000,0.000000,0.043478,0.0,0.0000,0.000000,0.000000,0.043478,0.000000,0.0,0.086957,0.000000,0.043478,0.043478,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.043478,0.043478,0.000000,0.0,0.0,0.043478,0.000000,0.086957,0.000000,0.086957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478

In [17]:
n_york_grouped.shape

(18, 104)

In [18]:
num_top_venues = 5

for hood in n_york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = n_york_grouped[n_york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor / Wilson Heights / Downsview North----
              venue  freq
0       Coffee Shop  0.10
1              Bank  0.10
2     Deli / Bodega  0.05
3  Sushi Restaurant  0.05
4    Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09
2         Coffee Shop  0.09
3          Restaurant  0.09
4    Greek Restaurant  0.04


----Don Mills----
                 venue  freq
0  Sporting Goods Shop  0.07
1           Restaurant  0.07
2     Asian Restaurant  0.07
3          Coffee Shop  0.07
4                  Gym  0.07


----Downsview----
            venue  freq
0            Park  0.12
1   Grocery Store  0.12
2  Baseball Field  0.06
3  Discount Store  0.06
4   Shopping Mall  0.06


----Fairview 

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = n_york_grouped['Neighborhood']

for ind in np.arange(n_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(n_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Middle Eastern Restaurant,Deli / Bodega,Pharmacy,Pizza Place,Ice Cream Shop,Bridal Shop,Restaurant,Sandwich Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,Bedford Park / Lawrence Manor East,Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Greek Restaurant,Grocery Store,Hobby Shop,Indian Restaurant,Juice Bar,Liquor Store
3,Don Mills,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
4,Downsview,Park,Grocery Store,Baseball Field,Liquor Store,Business Service,Discount Store,Hotel,Shopping Mall,Gym / Fitness Center,Home Service


In [21]:
# set number of clusters
kclusters = 4

n_york_grouped_clustering = n_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(n_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 2, 0, 0, 1, 2])

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

n_york_merged = n_york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
n_york_merged = n_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

n_york_merged # check the last columns!

,Unnamed: 0,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Intersection,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Food Truck,Food Court,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
2,3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Gift Shop,Vietnamese Restaurant,Accessories Store
3,7,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
4,10,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Park,Pizza Place,Pub,Japanese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
5,13,M3C,North York,Don Mills,43.725900,-79.340923,0.0,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
6,27,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Golf Course,Pool,Mediterranean Restaurant,Fast Food Restaurant,Dog Run,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
7,28,M3H,North York,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259,0.0,Coffee Shop,Bank,Middle Eastern Restaurant,Deli / Bodega,Pharmacy,Pizza Place,Ice Cream Shop,Bridal Shop,Restaurant,Sandwich Place
8,33,M2J,North York,Fairview / Henry Farm / Oriole,43.778517,-79.346556,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Japanese Restaurant,Bus Station,Bank,Food Court,Toy / Game Store
9,34,M3J,North York,Northwood Park / York University,43.767980,-79.487262,0.0,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Coffee Shop,Women's Store,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


In [23]:
n_york_merged = n_york_merged.dropna() 
n_york_merged

,Unnamed: 0,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Intersection,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Food Truck,Food Court,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
2,3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Gift Shop,Vietnamese Restaurant,Accessories Store
3,7,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
4,10,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Park,Pizza Place,Pub,Japanese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
5,13,M3C,North York,Don Mills,43.725900,-79.340923,0.0,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
6,27,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Golf Course,Pool,Mediterranean Restaurant,Fast Food Restaurant,Dog Run,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
7,28,M3H,North York,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259,0.0,Coffee Shop,Bank,Middle Eastern Restaurant,Deli / Bodega,Pharmacy,Pizza Place,Ice Cream Shop,Bridal Shop,Restaurant,Sandwich Place
8,33,M2J,North York,Fairview / Henry Farm / Oriole,43.778517,-79.346556,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Japanese Restaurant,Bus Station,Bank,Food Court,Toy / Game Store
9,34,M3J,North York,Northwood Park / York University,43.767980,-79.487262,0.0,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Coffee Shop,Women's Store,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


In [24]:
import numpy as np
import sys

np.set_printoptions(threshold=sys.maxsize)   
n_york_merged = n_york_merged.astype({'Cluster Labels': int})
n_york_merged

,Unnamed: 0,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Intersection,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Food Truck,Food Court,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
2,3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Gift Shop,Vietnamese Restaurant,Accessories Store
3,7,M3B,North York,Don Mills,43.745906,-79.352188,0,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
4,10,M6B,North York,Glencairn,43.709577,-79.445073,0,Park,Pizza Place,Pub,Japanese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
5,13,M3C,North York,Don Mills,43.725900,-79.340923,0,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
6,27,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Pool,Mediterranean Restaurant,Fast Food Restaurant,Dog Run,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
7,28,M3H,North York,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259,0,Coffee Shop,Bank,Middle Eastern Restaurant,Deli / Bodega,Pharmacy,Pizza Place,Ice Cream Shop,Bridal Shop,Restaurant,Sandwich Place
8,33,M2J,North York,Fairview / Henry Farm / Oriole,43.778517,-79.346556,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Japanese Restaurant,Bus Station,Bank,Food Court,Toy / Game Store
9,34,M3J,North York,Northwood Park / York University,43.767980,-79.487262,0,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Coffee Shop,Women's Store,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [26]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(n_york_merged['Latitude'], n_york_merged['Longitude'], n_york_merged['Neighborhood'], n_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

 
map_clusters